In [59]:
import nfts.dataset
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [60]:
# Connection to Data
os.listdir(r"C:\Users\Rick\OneDrive\Information studies\Master thesis")
DATASET_PATH = r"C:\Users\Rick\OneDrive\Information studies\Master thesis\nfts.sqlite"

### Load in data using SQL

In [61]:
ds = nfts.dataset.FromSQLite(DATASET_PATH)

### Explains the data

In [62]:
nfts.dataset.explain()


The Moonstream NFTs dataset

To load the NFTs dataset from a SQLite file, run:
>>> ds = nfts.dataset.FromSQLite(<path to sqlite database>)

This dataset consists of the following dataframes:

Dataframe: nfts
Load using:
	nfts_df = ds.load_dataframe(<sqlite connection or path to sqlite db>, "nfts")

Describes the NFT contracts represented in this dataset, with a name and symbol if they were available at time of crawl.

Columns:
1. address: The Ethereum address of the NFT contract.
2. name: The name of the collection of NFTs that the contract represents.
3. symbol: The symbol of the collection of NFTs that the contract represents.

- - -

Dataframe: mints
Load using:
	mints_df = ds.load_dataframe(<sqlite connection or path to sqlite db>, "mints")

All token mint events crawled in this dataset.

Columns:
1. event_id: A unique event ID associated with the event.
2. transaction_hash: The hash of the transaction which triggered the event.
3. block_number: The transaction block in which the 

### We are interested in the transaction data of NFT's

In [63]:
transactions = ds.load_dataframe("transfers")

In [64]:
random_transactions = transactions.sample(n=10000)

In [65]:
random_transactions.head()

,event_id,transaction_hash,block_number,nft_address,token_id,from_address,to_address,transaction_value,timestamp
2272042,edfc09f2-0708-4bd8-8125-48809f4b169b,0xc23c2f86ec69294c1f50f26baf6ba3b48a3576ef18b9...,13228378,0x71E3D77D79a6787b2d5783F1BE787188382960fb,1088,0xd98403EAD18CD4351813580aFCf2E231cD762309,0x5EBB53Ca3FAaDA5E46934c90cCBF7D79d50Ba9f8,2000000000000000.00000,1631683438
3748489,97c15b19-9e95-4a9f-a8ed-429c7481c09a,0x79019650e41ee9fcb3407395f7687821f5145a41d00e...,12879157,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,1163914,0x4886404ECFB86B759B27867AFB0F64172eeA1C53,0xb1690C08E213a35Ed9bAb7B318DE14420FB57d8C,0.00000,1626996178
2961864,ba6eee42-5af6-482a-9c3e-6222a5d18eb7,0xde65f6f15544b8b23883dba8dcb856cdb4d9989e1c8e...,13295543,0xB5C747561a185A146f83cFff25BdfD2455b31fF4,787,0x50591b0a8274DD51617E4762D8251CcbBDFfFf02,0x97a4E84a0c1945B0162B2A92f9f566B7Fd815f70,0.00000,1632580572
1748829,55845187-1d7e-452e-b096-ab3ece5d65f7,0xb3bf3f734a990b047a814442cd45c09b537543b2ecc6...,13165019,0x392179031da3012DaC321703A29E4c9fbd26316B,3282,0x23727EbdE0056732754Df6a75bDE03FcD8706F35,0x5F5140A6b5821bD505EA4D92b4Cd62136A1e24F6,0.00000,1630835527
3097462,300603f2-f776-408e-816a-6ccc022e16ab,0x517171e779c65d3d9666dc043193fe7aea3046911a47...,12630232,0x3B3ee1931Dc30C1957379FAc9aba94D1C48a5405,8274,0xfa22F5D75165111f2F6Ad5f8A43a7e3463e4Db96,0x2BeDc55Ced8ad734cF09e77003fed3DF641aAD3c,0.00000,1623642080


##### We are mostly interested in the adresses of the owners of the NFT's, the NFT itself (token_id), the transaction (event_id), the value of the transaction and the time it was sold. The blocknumbers are not saying a lot as well as the transaction_hash. (Only run once)

In [66]:
#delete unnecassey columns transaction_hash and block_number
random_transactions.drop(['transaction_hash', 'block_number'], axis = 1, inplace = True)

##### The column timestamp is changed to date so it is readible The column transaction_value is translated from WEI (smallest form of ETH) to ethereum for readibility purposes

In [67]:
#change timestamp to actual readible date and delete original column
random_transactions['date'] = pd.to_datetime(random_transactions['timestamp'],unit='s')
random_transactions.drop(['timestamp'], axis = 1, inplace = True)

#change the transaction value from WEI to ETH
random_transactions['transaction_value'] = random_transactions['transaction_value'] / 1000000000000000000

In [68]:
random_transactions.head()

,event_id,nft_address,token_id,from_address,to_address,transaction_value,date
2272042,edfc09f2-0708-4bd8-8125-48809f4b169b,0x71E3D77D79a6787b2d5783F1BE787188382960fb,1088,0xd98403EAD18CD4351813580aFCf2E231cD762309,0x5EBB53Ca3FAaDA5E46934c90cCBF7D79d50Ba9f8,0.00200,2021-09-15 05:23:58
3748489,97c15b19-9e95-4a9f-a8ed-429c7481c09a,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,1163914,0x4886404ECFB86B759B27867AFB0F64172eeA1C53,0xb1690C08E213a35Ed9bAb7B318DE14420FB57d8C,0.00000,2021-07-22 23:22:58
2961864,ba6eee42-5af6-482a-9c3e-6222a5d18eb7,0xB5C747561a185A146f83cFff25BdfD2455b31fF4,787,0x50591b0a8274DD51617E4762D8251CcbBDFfFf02,0x97a4E84a0c1945B0162B2A92f9f566B7Fd815f70,0.00000,2021-09-25 14:36:12
1748829,55845187-1d7e-452e-b096-ab3ece5d65f7,0x392179031da3012DaC321703A29E4c9fbd26316B,3282,0x23727EbdE0056732754Df6a75bDE03FcD8706F35,0x5F5140A6b5821bD505EA4D92b4Cd62136A1e24F6,0.00000,2021-09-05 09:52:07
3097462,300603f2-f776-408e-816a-6ccc022e16ab,0x3B3ee1931Dc30C1957379FAc9aba94D1C48a5405,8274,0xfa22F5D75165111f2F6Ad5f8A43a7e3463e4Db96,0x2BeDc55Ced8ad734cF09e77003fed3DF641aAD3c,0.00000,2021-06-14 03:41:20
